# Homework 1

## 1.1

Introspection is the process of analyzing one's own thoughts. This can be done either by thinking about your own thoughts or by asking others what they are thinking and why they are thinking those things. This method of studying the human mind was very ineffective because we are bad at interpreting what we are thinking and have no way of knowing what goes on in our subconscious because, by definition, it is impossible to access our subconscious from our conscious minds. 

In AI, we need a reliable model to base human mental processes off to get something artificially intelligent. Early researchers tried to use introspection to get this model, however, it is not a good way to do this because our cognitive processes are abstracted one layer above our subconscious and we have no way of interpreting how those subconscious processes take place by thinking about it. 

## 1.2

For the initial value in my implementation of the traveling salesman problem I am using a list of the cities 0 through n (the number of cities in the problem) and shuffling them to get a random start. The actions are defined by tuples representing all the possible swaps between cities on the route. The results come back as a list that has had a swap operation from the two cities in the action. The value function finds the distances that were randomly generated at the initialization of the problem in a symmetric matrix so that the distance to each city is the same both ways.

In [157]:
import math
import numpy as np
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import shuffle


class TspDefinition(Problem):

    def __init__(self, cities):
        self.cities = cities
        self.city_list = self.create_city_list(cities)
        
        # Matrix with the city distances
        self.city_dist = self.create_city_matrix(cities)
        
        # Initial value set to a shuffled copy of the city list
        self.initial = self.city_list
        shuffle(self.city_list)

    # Creates a list of all of the cities in the problem (cities 0 through n-1)
    def create_city_list(self, cities):
        city_list = []
        for n in range(cities):
            city_list.append(n)
        return city_list
        
    # Creates a symmetric matrix of random values, used as the distances between cities
    def create_city_matrix(self, cities):
        city_matrix = np.random.randint(1, 1000, size=(cities, cities))  # Create random matrix
        city_matrix_symm = (city_matrix + city_matrix.T)/2  # Makes the matrix symmetric
        return city_matrix_symm
    
    # Defines the possible actions for the problem
    def actions(self, state):
        all_actions = []
        
        # Iterates through all combinations of 2 cities and appends them as a tuple to the return matrix
        for city1 in range(self.cities):
            for city2 in range(self.cities - city1 - 1):
                action = (city1, city2 + city1 + 1)
                all_actions.append(action)
                
        return all_actions
        
    # Defines the result of a given action
    def result(self, x, action):
        new_x = x.copy() # Makes a copy of the state
        new_x[action[0]], new_x[action[1]] = new_x[action[1]], new_x[action[0]] # Swap the cities in the action tuple
        return new_x

    def value(self, x):
        total_distance = 0
        last_city = 0
        # Starting the with second city, find the distance in the matrix and add to the total distance
        for city in range(self.cities):
            if city != 0:
                total_distance -= self.city_dist[x[last_city]][x[city]]
                last_city = city
        # Adds the distance between the last and first city to make a loop
        total_distance += self.city_dist[x[-1]][x[0]]
        return total_distance


The city domain I am using for this problem can be an arbitrary number of cities that will have random distances assigned to them (they don’t make geometric sense). I am using the hill-climbing algorithm and the simulated annealing algorithm with 100 random restarts to see which one does better over a wider sample. Overall, the simulated annealing algorithm works better than hill climbing. This is because the solution space for TSPs is spread out, which lends its self to simulated annealing, however, the hill climbing algorithm finds a decently good solution as well.

In [158]:
print('Starting TSP problem')
if __name__ == '__main__':
    max_annealing = 0
    max_hill = 0
    initial_val = 0
    
    # Run hill-climing and simulated annealing with random restarts 100x
    for x in range(100):
        p = TspDefinition(7)
        initial_val += p.value(p.initial)
        hill_solution = hill_climbing(p)
 
        max_hill += p.value(hill_solution)
        # Solve the problem using simulated annealing.
        annealing_solution = simulated_annealing(
            p,
            exp_schedule(k=40, lam=.00005, limit=1000)
        )
        max_annealing += p.value(annealing_solution)
    
    # Print the average results of each algorithm
    print("Initial state average value: " + str(initial_val / 100))
    print('Hill climbing average value: ' + str(max_hill / 100))
    print('Simulated annealing average value: ' + str(max_annealing / 100))


Starting TSP problem


Initial state average value: -2529.4
Hill climbing average value: -1172.565
Simulated annealing average value: -1121.495


The travelling salesman problem was an interesting exercise in using local search and showed that simple algorithms can find decently good solutions for very large and np complete problems like this.

## 1.3

For the scheduling problem, I used the classes as variables and the time, room, and professor make up the domain which is all the different combinations of those. The constraints were that one class is in one room at one time and the professors assigned to each class is correct.

In [144]:
from collections.__init__ import defaultdict
from typing import Dict, List, Any

from csp import *


def class_schedule():

    times = 'mwf8000 mwf9000 mwf1030 th1330'.split()
    classrooms = 'nh253 sb382'.split()
    faculties = 'dschuurman jadams kvanderlinden hplantinga'.split()
    classes = 'CS108 CS112 CS212 CS214 CS344 CS300'.split()
    class_assignments = {'CS108': 'dschuurman', 'CS112': 'jadams', 'CS212': 'hplantinga', 'CS214': 'jadams'
                         , 'CS344': 'kvanderlinden', 'CS300': 'dschuurman'}
    
    domains = {}

    # Add all of the possibilities to the domain
    for class_ in classes:
        domains[class_] = list()
        for faculty in faculties:
            for time in times:
                for classroom in classrooms:
                    domains[class_].append([faculty, time, classroom])
                    
    # Parses the neighbors in a loop so they are all checked
    neighbors = parse_neighbors("""CS108: CS112;
                      CS112: CS212; CS212: CS214; CS214: CS344;
                      CS344: CS300; CS300: CS108""", classes)
    for type in [classes]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def schedule_constraints(A, a, B, b, recurse=0):
        # Checks if the professors are assigned incorrectly, ensures 1 course per prof
        if class_assignments[A] != a[0] and class_assignments[B] != b[0]:
            return False
        # Checks if the faculty member a/b[0] is teaching two courses at the same time
        if a[0] == b[0] and a[1] == b[1]:
            return False
        # Checks if the room a/b[2] is being used at the same time
        if a[1] == b[1] and a[2] == b[2]:
            return False
        return True  # Otherwise return true

    return CSP(classes, domains, neighbors, schedule_constraints)


In [146]:
from csp import min_conflicts, backtracking_search, AC3
from search import depth_first_graph_search


def print_solution(result):
    """A CSP solution printer copied from csp.py."""
    for h in ['CS108', 'CS112', 'CS212', 'CS214', 'CS344', 'CS300']:
        print('class', h)
        for (val, var) in result.items():
            if val == h: print('\t', var)


puzzle = class_schedule()

# result = depth_first_graph_search(puzzle)
# result = AC3(puzzle)
result = backtracking_search(puzzle)
# result = min_conflicts(puzzle, max_steps=1000)

if puzzle.goal_test(puzzle.infer_assignment()):
    print("Solution:\n")
    print_solution(result)
else:
    print("failed...")
    print(puzzle.curr_domains)
    puzzle.display(puzzle.infer_assignment())


Solution:

class CS108
	 ['dschuurman', 'mwf8000', 'nh253']
class CS112
	 ['jadams', 'mwf8000', 'sb382']
class CS212
	 ['hplantinga', 'mwf9000', 'nh253']
class CS214
	 ['jadams', 'mwf9000', 'sb382']
class CS344
	 ['kvanderlinden', 'mwf1030', 'nh253']
class CS300
	 ['dschuurman', 'mwf1030', 'sb382']


The scheduling problem shows that by just defining the problem that has many variables and constraints can be used to find a solution very quickly (provided you code faster than me).